# Scala Spark: Simulating Data

Simulating data is an important technique to:

1. Explore simple assumptions and how they play around statistically
2. Check the sense of your data processing and analysis


Suppose you build a complex data processing pipleline. How do you know that it's working? ie., not misprocessing the data. 

If you know ahead of time what the simulation will tell you, your analytical pipeline should produe these assumptions as its conclusions. 

It's can be a good idea to build a simulation of data sets *before* using real-world data. This allows you to think through the various attributes you will be provided, and allows you to build a data processing pipeline you can unit-test with known fixed outputs. 

## Discrete Random Numbers: Flips and Trials

* A 1/5 chance of picking true:

In [3]:
val choices = true +: Array.fill(4)(false)

choices: Array[Boolean] = Array(true, false, false, false, false)

* A random index:

In [5]:
import scala.util.Random

Random.nextInt(choices.length)

import scala.util.Random


res4_1: Int = 1

* A random choice, true with prb. 20% ...

In [6]:
choices(Random.nextInt(choices.length))

res5: Boolean = false

* Let's make these methods:

In [8]:
object Choose {
    def pick[A](choices: Seq[A]) = choices(Random.nextInt(choices.length))
    def flip(odds: Int) = pick(true +: Array.fill(odds - 1)(false))
}


defined object Choose

In [9]:
Choose.flip(2) // prb. 1 in 2 , ie., 50%

res8: Boolean = false

* 10 flips:

In [10]:
Array.fill(10)(Choose.flip(2))

res9: Array[Boolean] = Array(
  true,
  true,
  false,
  true,
  false,
  false,
  true,
  false,
  true,
  true
)

#### Illustrative uses

In [13]:
Array.fill(10)(Choose.flip(2)).count( _ == true ) 

res12: Int = 6

In [14]:
Array.fill(1000)(Choose.flip(2)).count( _ == true) / 1000.0

res13: Double = 0.518

In [17]:
Array.fill(1000)(Choose.flip(3)).count( _ == true) / 1000.0

res16: Double = 0.337

In [18]:
Choose.pick(18 to 30)

res17: Int = 25

In [19]:
Choose.pick(List("London", "Leeds"))

res18: String = "London"

In [20]:
Choose.pick(List("Sherlock", "Watson"))

res19: String = "Sherlock"

## Continous Random Numbers: The Normal Distribution

In [24]:
val knownMean = 10

knownMean: Int = 10

In [25]:
val knownSD = 5

knownSD: Int = 5

* Five random numbers with standard deviation knownSD and mean knownMean

In [22]:
new java.util.Random().nextGaussian()

res21: Double = 0.977131313671385

In [23]:
val r = new java.util.Random()

r: Random = java.util.Random@5c0a7148

In [26]:
Vector.fill(5)(knownSD * r.nextGaussian() + knownMean)

res25: Vector[Double] = Vector(
  10.491342385193198,
  15.85099201249532,
  17.135115394351068,
  13.073481748942566,
  15.122401291768236
)

* Let's calculate mean, varience and std. deviation of 1000 such numbers:

In [27]:
object Normal {
    val _r = new java.util.Random()
    val normals = Vector.fill(1000)(knownSD * _r.nextGaussian() + knownMean) 
    
    
    val total_ = normals.reduce( _ + _ )
    val mean_ = total_ / normals.length
    val var_ = normals.map( _ - mean_ ).map(math.pow(_, 2)).reduce(_ + _) / normals.length
    val sd_ = math.sqrt(var_)
    
}

defined object Normal

* The sample of 1000 has approximately the known mean and known std. dev.

In [28]:
Normal.sd_

res27: Double = 5.204307819337279

In [29]:
Normal.mean_

res28: Double = 9.840070049726146

* Let's package this up into something useful:

In [35]:
object Choose {
    val _r = new java.util.Random()
    
    def pick[A](choices: Seq[A]) = choices(_r.nextInt(choices.length))
    def flip(odds: Int) = pick(true +: Array.fill(odds - 1)(false))
    
    
    def around(point: Double, within: Double) = pick(Vector.fill(10)(within * _r.nextGaussian() + point) )
}




defined object Choose

In [36]:
Choose.around(10, 2)

res35: Double = 9.964221743283609

## A Data Generator

* Suppose we want to generate many objects:

In [37]:
case class Product(
    level: String,
    price: Double,
    weight: Double,
    isElectrical: Boolean,
    isDomestic: Boolean,
    isProfessional: Boolean
)

defined class Product

* Some fields are discrete (small set of options); others are continous (number falls anywhere in range). 
* Let's define a Generator to generate a Product:

In [38]:
object Generator {
    val options = Array("Gold", "Silver", "Bronze")
    val prices = 0 to 10
    
    def genProduct() = {
        val isExpensive = flip(10)
        
        Product(
            level  = Choose.pick(options),
            price  = Choose.pick(prices) * (if(isExpensive) 10 else 1),
            weight = Choose.around(10, 2),
            isElectrical = Choose.flip(2),
            isDomestic = Choose.flip(3),
            isProfessional = (isExpensive || Choose.flip(5)),
        )
    }
    
    
    def genMany(amount: Int) = Array.fill(amount) { genProduct() }
}

defined object Generator

In [39]:
Generator.genMany(2)

res38: Array[Product] = Array(
  Product("Gold", 6.0, 10.794630306972964, false, true, false),
  Product("Gold", 10.0, 9.380459485937807, true, false, false)
)

* Let's test the generator by calculating various statistical measures and checking for sense:

In [40]:
object Product {
    val weights = Generator.genMany(100).map( _.weight )
    val meanWeight_ = weights.reduce( _ + _ ) / weights.length
    
    val isPro = Generator.genMany(100).map( _.isProfessional )
    val probPro_ = isPro.count( _ == true) / isPro.length.toDouble
}

defined object Product

In [77]:
Product.meanWeight_

res76: Double = 10.31417706806951

In [41]:
Product.probPro_

res40: Double = 0.33

* Notice that prob. of being professional is c. 0.3 which is c. 0.2 + 0.1
    * why?
    * HINT: when do probabilities add? when do they multiply?